##### Copyright 2018 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Save and restore models

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/save_and_restore_models"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/models/blob/master/samples/core/tutorials/keras/save_and_restore_models.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/models/blob/master/samples/core/tutorials/keras/save_and_restore_models.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Model progress can be saved during—and after—training. This means a **model can resume where it left off and avoid long training times. Saving also means you can share your model** and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share:

* code to create the model, and
* the trained weights, or parameters, for the model

Sharing this data helps others understand how the model works and try it themselves with new data.

**Caution: Be careful with untrusted code—TensorFlow models are code. See [Using TensorFlow Securely](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md) for details.**

### Options

There are different ways to save TensorFlow models—depending on the API you're using. This guide uses [tf.keras](https://www.tensorflow.org/guide/keras), a high-level API to build and train models in TensorFlow. For other approaches, see the TensorFlow  [Save and Restore](https://www.tensorflow.org/guide/saved_model) guide or [Saving in eager](https://www.tensorflow.org/guide/eager#object_based_saving).


## Setup

### Installs and imports

Install and import TensorFlow and dependencies:

In [3]:
!pip install h5py pyyaml 

### Get an example dataset

We'll use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to train our model to demonstrate saving weights. To speed up these demonstration runs, only use the first 1000 examples:

In [4]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__
print('Using CUDA: %s' % tf.test.is_built_with_cuda())

# check jupyter notebook console output to ensure that we are using GPU
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

Using CUDA: True


In [5]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

SAMPLE_SIZE = 1000
ASSUMED_IMAGE_DIM = 28

train_labels = train_labels[:SAMPLE_SIZE]
test_labels = test_labels[:SAMPLE_SIZE]

print(train_images.shape)

def vectorize_and_normalize(images_matrix):
    GREYSCALE_MAX = 255.0
    return images_matrix.reshape(-1, ASSUMED_IMAGE_DIM**2) / GREYSCALE_MAX

train_images = vectorize_and_normalize(train_images[:SAMPLE_SIZE])
test_images = vectorize_and_normalize(test_images[:SAMPLE_SIZE])

print(train_images.shape)
print(train_images[0])

(60000, 28, 28)
(1000, 784)
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.   

### Define a model

Let's build a simple model we'll use to demonstrate saving and loading weights.

In [6]:
# Returns a short sequential model
def create_model():
    probability_dist_layer = keras.layers.Dense(10, activation=tf.nn.softmax)
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(train_images.shape[1],)),
        keras.layers.Dropout(0.2),
        probability_dist_layer
    ])
  
    model.compile(optimizer=tf.keras.optimizers.Adam(), 
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  
    return model


# Create a basic model instance
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training

**The primary use case is to automatically save checkpoints *during* and at *the end* of training**. This way you can use a trained model without having to retrain it, or pick-up training where you left of—in case the training process was interrupted.

`tf.keras.callbacks.ModelCheckpoint` is a callback that performs this task. The callback takes a couple of arguments to configure checkpointing.

### Checkpoint callback usage

Train the model and pass it the `ModelCheckpoint` callback:

In [7]:
checkpoint_path = "training_1/cp.ckpt" # relative to location of this file
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  
          epochs = 10, 
          validation_data = (test_images, test_labels),
          callbacks = [cp_callback])  # pass callback to training

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 0s 456us/step - loss: 1.1910 - acc: 0.6550 - val_loss: 0.7016 - val_acc: 0.8010

Epoch 00001: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
Epoch 2/10
1000/1000 [==============================] - 0s 87us/step - loss: 0.4279 - acc: 0.8900 - val_loss: 0.5208 - val_acc: 0.8390

Epoch 00002: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
Epoch 3/10
1000/1000 [==============================] - 0s 94us/step - loss: 0.2827 - acc: 0.9310 - val_loss: 0.4794 - val_acc: 0.8520

Epoch 00003: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
Epoch 4/10
1000/1000 [==============================] - 0s 90us/step - loss: 0.2064 - acc: 0.9470 - val_loss: 0.4359 - val_acc: 0.8590

Epoch 00004: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:

In [8]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


Create a new, untrained model. **When restoring a model from only weights, you must have a model with the same architecture as the original model**. Since it's the same model architecture, we **can share weights** despite that it's a different *instance* of the model.

Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [9]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 127us/step
Untrained model, accuracy:  8.80%


Then load the weights from the checkpoint, and re-evaluate:

In [10]:
model.load_weights(checkpoint_path)

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 29us/step
Restored model, accuracy: 86.30%


### Checkpoint callback options

The callback provides several options to give the resulting checkpoints unique names, and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every 5-epochs:


In [11]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt" # relative to this file location
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, 
          callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00010: saving model to training_2/cp-0010.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00015: saving model to training_2/cp-0015.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00020: saving model to training_2/cp-0020.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00025: saving model to training_2/cp-0025.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00030: saving model to training_2/cp-0030.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00035: saving model to training_2/cp-0035.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00040: saving model to training_2/cp-0040.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00045: saving model to training_2/cp-0045.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 000

Now, have a look at the resulting checkpoints (sorting by modification date):

In [17]:
import pathlib

# Sort the checkpoints by modification time.
checkpoints = pathlib.Path(checkpoint_dir).glob("*.index")
checkpoints = sorted(checkpoints, key=lambda chkpt:chkpt.stat().st_mtime)
checkpoints = [chkpt.with_suffix('') for chkpt in checkpoints]
latest = str(checkpoints[-1])

print(latest)
print(checkpoints)

training_2/cp-0050.ckpt
[PosixPath('training_2/cp-0030.ckpt'), PosixPath('training_2/cp-0035.ckpt'), PosixPath('training_2/cp-0040.ckpt'), PosixPath('training_2/cp-0045.ckpt'), PosixPath('training_2/cp-0050.ckpt')]


Note: the default tensorflow format only saves the 5 most recent checkpoints.

To test, reset the model and load the latest checkpoint:

In [18]:
model = create_model()
model.load_weights(latest) # so now we don't need to train these models to get the same optimizations

loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 98us/step
Restored model, accuracy: 87.40%


## What are these files?

The above code stores the weights to a collection of [checkpoint](https://www.tensorflow.org/guide/saved_model#save_and_restore_variables)-formatted files that **contain only the trained weights in a binary format**. Checkpoints contain:
* One or more shards that contain your model's weights. 
* An index file that indicates which weights are stored in a which shard.  

If you are only training a model on a single machine (as opposed to a tensorflow cluster), you'll have one shard with the suffix: `.data-00000-of-00001`

## Manually save weights

Above you saw how to load the weights into a model.

Manually saving the weights is just as simple, use the `Model.save_weights` method.

In [19]:
# Save the weights manually
wieght_checkpoint_location = "./checkpoints/my_checkpoint"
model.save_weights(wieght_checkpoint_location)

# Restore the weights
model = create_model()
model.load_weights(wieght_checkpoint_location)

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 0s 101us/step
Restored model, accuracy: 87.40%


## Save the entire model

The **entire model** can be saved to a file that contains the weight values, the model's configuration, and even the optimizer's configuration. This allows you to checkpoint a model and resume training later—from the exact same state—without access to the original code.

Saving a fully-functional model in Keras is very useful—you can load them in [TensorFlow.js](https://js.tensorflow.org/tutorials/import-keras.html) and then train and run them in web browsers.

Keras provides a basic save format using the [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard. For our purposes, the saved model **can be treated as a single binary blob** (see https://en.wikipedia.org/wiki/Binary_large_object).


In [20]:
model = create_model()

model.fit(train_images, train_labels, 
          epochs=5)

# Save entire model to a HDF5 file
model_save_location = 'my_model.h5'
model.save(model_save_location)

Epoch 1/5
1000/1000 [==============================] - 0s 350us/step - loss: 1.2261 - acc: 0.6300
Epoch 2/5
1000/1000 [==============================] - 0s 65us/step - loss: 0.4325 - acc: 0.8810
Epoch 3/5
1000/1000 [==============================] - 0s 73us/step - loss: 0.3037 - acc: 0.9270
Epoch 4/5
1000/1000 [==============================] - 0s 75us/step - loss: 0.2206 - acc: 0.9420
Epoch 5/5
1000/1000 [==============================] - 0s 68us/step - loss: 0.1557 - acc: 0.9680


Now recreate the model from that file:

In [21]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model(model_save_location)
new_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Check its accuracy:

In [22]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 128us/step
Restored model, accuracy: 86.80%


This technique saves everything:

* The weight values
* The model's configuration(architecture)
* The optimizer configuration

Keras saves models by inspecting the architecture. Currently, it is not able to save TensorFlow optimizers (from `tf.train`). When using those you will need to re-compile the model after loading, and you will loose the state of the optimizer.


## What's Next

That was a quick guide to saving and loading in with `tf.keras`.

* The [tf.keras guide](https://www.tensorflow.org/guide/keras) shows more about saving and loading models with `tf.keras`.

* See [Saving in eager](https://www.tensorflow.org/guide/eager#object_based_saving) for saving during eager execution.

* The [Save and Restore](https://www.tensorflow.org/guide/saved_model) guide has low-level details about TensorFlow saving.